In [214]:
.ORIG x3000
LD R0, START_ADDRESS
JMP R0

START_ADDRESS: .FILL START

;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;;;------------------------------------------------------
START: 

    LEA R6, D16
    JSR PUSH
    
    LD R0, D16_roll
    OUT
    HALT

;;; -----------------------------------------------------
;;; After this runs, result in D16_roll
D16: 
    ;; Call Random, loading a new random number into X
    LEA R6, Random
    JSR Push
    
    LD R0, X
    AND R0, R0, #15
    ADD R0, R0, #1
    ST R0, D16_roll
    
    LEA R0, POP
    JMP R0
D16_roll: .FILL #0


;;; -----------------------------------------------------
;;; Memory X has next random number
Random: 
    LD R0, M
    LD R1, A
    
    LEA R6, Divide
    JSR PUSH         ; R0 / R1
    
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    
    LEA R6, Divide
    JSR PUSH        ; R0 / R1
    
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    
    LEA R6, Multiply
    JSR PUSH        
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1
    
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    
    LEA R6, Multiply
    JSR PUSH            
    LD R2, MULT_PRODUCT ; R2 <- r * TEMP2
    
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
    
TEST:  BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: ST R2, X
    LEA R0, POP         ;; Return
    JMP R0

A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0
TEMP1: .FILL #0
TEMP2: .FILL #0
BACK: .FILL #0

;;; -----------------------------------------------------
;;; Mult_Product <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:
  ST R2, MULT_PRODUCT           ; Product has the product
  LEA R0, POP
  JMP R0
  
MULT_PRODUCT: .FILL #0

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  LEA R0, POP
  JMP R0
QUOTIENT: .FILL #0
REMAINDER: .FILL #0
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [230]:
%exe

============================================================
Computation completed
Instructions: 33137
Cycles: 232128 (0.116064 milliseconds)

Registers:
PC: x048E
N: 0 Z: 1 P: 0 
R0: x0004 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x3004 R6: x308F R7: x308F 


In [186]:
%dis 30A4

Memory disassembled:
           x30A4: x225D  LD R1, QUOTIENT                           [line: 96]
           x30A5: x321D  ST R1, TEMP2                              [line: 97]
           x30A6: x225C  LD R1, REMAINDER                          [line: 98]
           x30A7: x321A  ST R1, TEMP1                              [line: 99]
           x30A8: x2014  LD R0, A                                  [line: 101]
           x30A9: xE01B  LEA R0, MULTIPLY                          [line: 103]
           x30AA: x4FBD  JSR PUSH                                  [line: 104]
           x30AB: x2433  LD R2, MULT_PRODUCT                       [line: 105]
           x30AC: x3415  ST R2, TEMP1                              [line: 107]
           x30AD: x2012  LD R0, R                                  [line: 109]
           x30AE: x2214  LD R1, TEMP2                              [line: 110]
           x30AF: xE015  LEA R0, MULTIPLY                          [line: 112]
           x30B0: x4FB7  JSR PUSH  